# Ensemble Evaluation: Timepoint 1

Location: New York State

Timepoint 1: April 3, 2020. Setting: New York State at the beginning of the pandemic when masking was the main preventative measure. No vaccines available.

For each timepoint, consider the following:
 - What is the most relevant data to use for model calibration?
 - What was our understanding of COVID-19 viral mechanisms at the time? For example, early in the pandemic, we didn't know if reinfection was a common occurance, or even possible.
 - What are the parameters related to contagiousness/transmissibility and severity of the dominant strain at the time?
 - What policies were in place for a stated location, and how can this information be incorporated into models?

For each timepoint:

1. (a) Take a single model, calibrate it using any historical data prior to the given date, and create a 4-week forecast for cases, hospitalizations, and deaths beginning on the given date. (b) Evaluate the forecast using the COVID-19 Forecasting Hub Error Metrics (WIS, MAE). The single model evaluation should be done in the same way as the ensemble.

2. Repeat (1), but with an ensemble of different models.

a. It is fine to calibrate each model independently and weight naively.

b. It would also be fine to calibrate the ensemble as a whole, assigning weights to the different component models, so that you minimize the error of the ensemble vs. historical data.

c. Use the calibration scores and error metrics computed by the CDC Forecasting Hub. As stated on their website:

“Periodically, we evaluate the accuracy and precision of the ensemble forecast and component models over recent and historical forecasting periods. Models forecasting incident hospitalizations at a national and state level are evaluated using adjusted relative weighted interval scores (WIS, a measure of distributional accuracy), and adjusted relative mean absolute error (MAE), and calibration scores. Scores are evaluated across weeks, locations, and targets. You can read a paper explaining these procedures in more detail, and look at the most recent monthly evaluation reports. The final report that includes case and death forecast evaluations is 2023-03-13.”

3. Produce the forecast outputs in the format specified by the CDC forecasting challenge, including the specified quantiles.

### Load dependencies

In [1]:
import os
import pandas as pd
import numpy as np
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_and_optimize_and_sample_petri_model,
    load_and_calibrate_and_optimize_and_sample_petri_model
)
from pyciemss.visuals import plots

## Get data

In [21]:
url = 'https://raw.githubusercontent.com/DARPA-ASKEM/experiments/main/thin-thread-examples/milestone_12month/evaluation/ensemble_eval_SA/datasets/aabb3684-a7ea-4f60-98f1-a8e673ad6df5/dataset.csv'
ny_data = pd.read_csv(url)

# Select historical data through Timepoint 1: 04/03/2020 (the first 72 rows)
# No hospitalization data at this point
ny_data = ny_data[0:73]
ny_data[["I", "H", "D"]].to_csv("NY_data1.csv")

## Select relevant models

In [3]:
model1_location = "../../notebook/ensemble_eval_sa/SEIRHDS_basic_deterministic.json"
model2_location = "../../notebook/ensemble_eval_sa/SEIRHDS_basic_config.json"
model3_location = "../../notebook/ensemble_eval_sa/SIRHD_age_stratified.json"
model4_location = "../../notebook/ensemble_eval_sa/SIRHD_mask_V1.json"
model5_location = "../../notebook/ensemble_eval_sa/SIRHD_mask_V2.json"
model6_location = "../../notebook/ensemble_eval_sa/SIRHD_mask_V3.json"
model7_location = "../../notebook/ensemble_eval_sa/SIRHD_V1C.json"

### Load and sample selected models

In [11]:
num_samples = 3
start_timepoint = 0
stop_timepoint = 72 + 4*7 # simulate for four weeks after end of data
timepoints = [float(i) for i in range(stop_timepoint + 1)]
result = load_and_sample_petri_model(model1_location, num_samples, timepoints=timepoints, time_unit="days", visual_options={"title": "SIR Model", "subset": ".*_sol"})
plots.ipy_display(result["visual"])

In [8]:
result = load_and_sample_petri_model(model4_location, num_samples, timepoints=timepoints, time_unit="days", visual_options={"title": "SIR Model", "subset": ".*_sol"})
plots.ipy_display(result["visual"])

In [16]:
result = load_and_sample_petri_model(model6_location, num_samples, timepoints=timepoints, time_unit="days", visual_options={"title": "SIR Model", "subset": ".*_sol"})
plots.ipy_display(result["visual"])

### Load and sample an ensemble of one

In [39]:
weights = [1]
num_samples = 100
solution_mappings = [{"I": "I", "H": "H", "D": "D"}]

# Run sampling
result = load_and_sample_petri_ensemble(
    [model1_location], weights, solution_mappings, num_samples, timepoints, 
    time_unit="days",
    visual_options={"subset":".*_sol"}
)
plots.ipy_display(result["visual"])

### Load and calibrate and sample an ensemble of one

In [40]:
num_samples = 100
model_paths = [model4_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1]
solution_mappings = [{"I": "I", "H": "H", "D": "D"}]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=1000,
    num_iterations=10,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

plots.ipy_display(result["visual"])

iteration 0: loss = 5522.285171806812



### Load and calibrate and sample an ensemble of models

In [34]:
num_samples = 2
model_paths = [model1_location, model4_location, model6_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1/3, 1/3, 1/3]
solution_mappings = [{"I": "I", "H": "H", "D": "D"},
                     {"I": "I", "H": "H", "D": "D"}, 
                     {"I": "Cases", "H": "Hosp", "D": "Deaths"},
                     ]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=10,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

plots.ipy_display(result["visual"])

iteration 0: loss = 4155.41032075882



In [42]:
num_samples = 2
model_paths = [model1_location, model4_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1/2, 1/2]
solution_mappings = [{"I": "I", "H": "H", "D": "D"},
                     {"I": "I", "H": "H", "D": "D"}, 
                     ]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=10,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

plots.ipy_display(result["visual"])

iteration 0: loss = 2198.888832986355



In [36]:
num_samples = 2
model_paths = [model1_location, model4_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1/2, 1/2]
solution_mappings = [{"I": "I", "H": "H", "D": "D"},
                     {"I": "I", "H": "H", "D": "D"}, 
                     ]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=1.0,
    num_iterations=10,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

plots.ipy_display(result["visual"])

iteration 0: loss = 15395.453054904938



In [37]:
num_samples = 2
model_paths = [model1_location, model4_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1/2, 1/2]
solution_mappings = [{"I": "I", "H": "H", "D": "D"},
                     {"I": "I", "H": "H", "D": "D"}, 
                     ]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=100.0,
    num_iterations=10,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

plots.ipy_display(result["visual"])

iteration 0: loss = 6492.609392762184



## Simulate models

## Calibrate an ensemble of one for each model

### Model 1

### Model 2

### Model 3

## Calibrate an ensemble of all three models